In [3]:
import numpy as np

# from keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Dropout, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.preprocessing import image

from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from tensorflow.keras.applications.imagenet_utils import preprocess_input

#import pydot
from IPython.display import SVG

from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.utils import plot_model
#from resnets_utils import *
from tensorflow.keras.initializers import glorot_uniform

import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
from gliomi import *

import tensorflow.keras.backend as K

K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [4]:
from tensorflow.keras.backend import reshape

In [5]:
from tensorflow.keras.applications import *

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout

from keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras.optimizers import SGD, Adam

import tensorflow.keras.backend as K

In [6]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    print("identity_block", X)

    return X

In [7]:


def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)


    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    ##### SHORTCUT PATH #### (≈2 lines)
    #X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
    #                    kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    #X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    #X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    print("convolutional_block", X)
    
    return X



In [8]:
def ResNet50(input_shape=(224,224,1), classes=2):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(2), kernel_initializer = glorot_uniform(seed=0))(X)
    
    print("Model", X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [9]:
import tensorflow as tf

optimizers = {
    'rmsprop': (lambda : tf.keras.optimizers.RMSprop(learning_rate=0.0001)),
    'sgd': (lambda : tf.keras.optimizers.SGD(lr=0.001, momentum=0.9, decay=0.001/10, nesterov=False)),
    'adam' : (lambda : tf.keras.optimizers.Adam(lr=0.0001))
}

In [10]:
optimizer=optimizers['rmsprop']()


In [39]:
def evaluate_model(dataset, epochs=200, batch_sizes=16):
    
    #scores = []
    
    X_train, y_train, X_test, y_test = dataset_loader.get_split_categorical(test_size=0.2, random_state=42)
    
    model = ResNet50(input_shape = (224,224,1), classes=2)
    
    model.summary()
    
    model.compile(optimizer=optimizers['sgd'](), loss='categorical_crossentropy', metrics=['accuracy','binary_crossentropy'])
    
    print("Starting fit")
    
    fit = model.fit(
            X_train, y_train, 
            epochs=epochs,
            validation_data=(X_test, y_test), 
            batch_size=batch_sizes)

    score = model.evaluate(X_test, y_test)[1]

    #print(random_state, ":", score)

    #scores.append(score)

    return np.array(score)

In [19]:
def count_created_data(dataset, epochs=200, batch_sizes=16):
    X_train, y_train, X_test, y_test = dataset_loader.get_split_categorical(test_size=0.2, random_state=42)
    
    aug = ImageDataGenerator(rotation_range=90,
                             zoom_range=0.15,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.15,
                             fill_mode="nearest")
    
    print("Batch:", batch_sizes)
    print("X shape", X_train.shape)
    
    for i in aug.flow(X_train, y_train, batch_size=batch_sizes):
        #print(len(i),i[0].shape)
        
        zero=np.where(y_train==0)[0].shape
        one=np.where(y_train==1)[0].shape
        
        print(zero, one)
        

In [63]:
def evaluate_model_data_augmentation(dataset, epochs=200, batch_sizes=16):
    
    #scores = []
    
    X_train, y_train, X_test, y_test = dataset_loader.get_split_categorical(test_size=0.2, random_state=42)
    
    aug = ImageDataGenerator(rotation_range=90,
                             zoom_range=0.15,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.15,
                             fill_mode="nearest")
    
    model = ResNet50(input_shape = (224,224,1), classes=2)
    
    model.summary()
    
    model.compile(optimizer=optimizers['sgd'](), loss='categorical_crossentropy', metrics=['accuracy','binary_crossentropy'])
    
    print("Starting fit")
    
    fit = model.fit(
            x=aug.flow(X_train, y_train, batch_size=batch_sizes), 
            epochs=epochs,
            validation_data=(X_test, y_test))
    

    
    train_score = model.evaluate(X_train, y_train)
    
    test_score = model.evaluate(X_test, y_test)

    #print(random_state, ":", score)

    #scores.append(score)

    return train_score, test_score

In [11]:
slice_dataset="/data/RMN/dataset-gliomi-cnn/datasets-tumor-crop"

dataset="mgmt"

sequence="flair"

percentile=100

side=224

epochs=500

batch_sizes=32

In [65]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
dataset_loader = DatasetLoader(f"{slice_dataset}/dataset-{dataset}-{sequence}-{side}-{percentile}-perc.pickle")

train_score, test_score = evaluate_model_data_augmentation(dataset_loader, epochs=epochs, batch_sizes=16)

#scores_str = ", ".join(str(x) for x in (scores * 100))

convolutional_block Tensor("activation_395/Relu:0", shape=(?, 55, 55, 256), dtype=float32)
identity_block Tensor("activation_398/Relu:0", shape=(?, 55, 55, 256), dtype=float32)
identity_block Tensor("activation_401/Relu:0", shape=(?, 55, 55, 256), dtype=float32)
convolutional_block Tensor("activation_404/Relu:0", shape=(?, 28, 28, 512), dtype=float32)
identity_block Tensor("activation_407/Relu:0", shape=(?, 28, 28, 512), dtype=float32)
identity_block Tensor("activation_410/Relu:0", shape=(?, 28, 28, 512), dtype=float32)
identity_block Tensor("activation_413/Relu:0", shape=(?, 28, 28, 512), dtype=float32)
convolutional_block Tensor("activation_416/Relu:0", shape=(?, 14, 14, 1024), dtype=float32)
identity_block Tensor("activation_419/Relu:0", shape=(?, 14, 14, 1024), dtype=float32)
identity_block Tensor("activation_422/Relu:0", shape=(?, 14, 14, 1024), dtype=float32)
identity_block Tensor("activation_425/Relu:0", shape=(?, 14, 14, 1024), dtype=float32)
identity_block Tensor("activation_4

res3a_branch2c (Conv2D)         (None, 28, 28, 512)  66048       activation_403[0][0]             
__________________________________________________________________________________________________
bn3a_branch2c (BatchNormalizati (None, 28, 28, 512)  2048        res3a_branch2c[0][0]             
__________________________________________________________________________________________________
activation_404 (Activation)     (None, 28, 28, 512)  0           bn3a_branch2c[0][0]              
__________________________________________________________________________________________________
res3b_branch2a (Conv2D)         (None, 28, 28, 128)  65664       activation_404[0][0]             
__________________________________________________________________________________________________
bn3b_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3b_branch2a[0][0]             
__________________________________________________________________________________________________
activation

Epoch 1/500
6/6 [==============================] - 16s 3s/step - loss: 3.0002 - acc: 0.5104 - binary_crossentropy: 2.5944 - val_loss: 7.1293 - val_acc: 0.4583 - val_binary_crossentropy: 5.0561
Epoch 2/500
6/6 [==============================] - 1s 178ms/step - loss: 3.0578 - acc: 0.5104 - binary_crossentropy: 2.9738 - val_loss: 3.1870 - val_acc: 0.6250 - val_binary_crossentropy: 2.9085
Epoch 3/500
6/6 [==============================] - 1s 172ms/step - loss: 6.5222 - acc: 0.5625 - binary_crossentropy: 3.4954 - val_loss: 0.8891 - val_acc: 0.7083 - val_binary_crossentropy: 0.9886
Epoch 4/500
6/6 [==============================] - 1s 172ms/step - loss: 5.0543 - acc: 0.4896 - binary_crossentropy: 3.8745 - val_loss: 3.4400 - val_acc: 0.5833 - val_binary_crossentropy: 2.9195
Epoch 5/500
6/6 [==============================] - 1s 171ms/step - loss: 5.0302 - acc: 0.5312 - binary_crossentropy: 2.9506 - val_loss: 2.5811 - val_acc: 0.5417 - val_binary_crossentropy: 2.5788
Epoch 6/500
6/6 [==========

Epoch 17/500
6/6 [==============================] - 1s 171ms/step - loss: 2.3467 - acc: 0.5104 - binary_crossentropy: 1.8315 - val_loss: 4.0039 - val_acc: 0.6250 - val_binary_crossentropy: 2.2505
Epoch 18/500
6/6 [==============================] - 1s 173ms/step - loss: 1.6888 - acc: 0.5208 - binary_crossentropy: 1.6878 - val_loss: 5.0286 - val_acc: 0.3750 - val_binary_crossentropy: 2.5587
Epoch 19/500
6/6 [==============================] - 1s 171ms/step - loss: 1.3011 - acc: 0.5833 - binary_crossentropy: 1.2922 - val_loss: 4.8368 - val_acc: 0.3750 - val_binary_crossentropy: 2.3691
Epoch 20/500
6/6 [==============================] - 1s 170ms/step - loss: 1.9860 - acc: 0.5833 - binary_crossentropy: 1.4741 - val_loss: 4.4288 - val_acc: 0.6250 - val_binary_crossentropy: 2.1400
Epoch 21/500
6/6 [==============================] - 1s 172ms/step - loss: 0.9746 - acc: 0.5417 - binary_crossentropy: 0.9746 - val_loss: 3.4603 - val_acc: 0.2917 - val_binary_crossentropy: 2.4379
Epoch 22/500
6/6 [==

Epoch 33/500
6/6 [==============================] - 1s 174ms/step - loss: 0.9361 - acc: 0.5312 - binary_crossentropy: 0.9361 - val_loss: 2.9583 - val_acc: 0.3750 - val_binary_crossentropy: 1.8321
Epoch 34/500
6/6 [==============================] - 1s 172ms/step - loss: 0.7366 - acc: 0.5938 - binary_crossentropy: 0.7366 - val_loss: 2.8611 - val_acc: 0.3750 - val_binary_crossentropy: 1.8502
Epoch 35/500
6/6 [==============================] - 1s 171ms/step - loss: 0.7405 - acc: 0.6042 - binary_crossentropy: 0.7405 - val_loss: 2.9112 - val_acc: 0.6667 - val_binary_crossentropy: 1.8726
Epoch 36/500
6/6 [==============================] - 1s 170ms/step - loss: 0.7523 - acc: 0.6771 - binary_crossentropy: 0.7523 - val_loss: 4.9324 - val_acc: 0.3333 - val_binary_crossentropy: 2.3419
Epoch 37/500
6/6 [==============================] - 1s 171ms/step - loss: 0.7621 - acc: 0.5833 - binary_crossentropy: 0.7621 - val_loss: 4.0042 - val_acc: 0.6250 - val_binary_crossentropy: 1.9752
Epoch 38/500
6/6 [==

...

Epoch 433/500
6/6 [==============================] - 1s 177ms/step - loss: 0.0829 - acc: 0.9583 - binary_crossentropy: 0.0829 - val_loss: 1.7611 - val_acc: 0.4583 - val_binary_crossentropy: 1.8259
Epoch 434/500
6/6 [==============================] - 1s 178ms/step - loss: 0.0881 - acc: 0.9792 - binary_crossentropy: 0.0881 - val_loss: 1.8059 - val_acc: 0.4583 - val_binary_crossentropy: 1.9138
Epoch 435/500
6/6 [==============================] - 1s 177ms/step - loss: 0.0311 - acc: 0.9896 - binary_crossentropy: 0.0311 - val_loss: 1.7899 - val_acc: 0.5000 - val_binary_crossentropy: 1.8902
Epoch 436/500
6/6 [==============================] - 1s 176ms/step - loss: 0.1337 - acc: 0.9479 - binary_crossentropy: 0.1337 - val_loss: 1.8904 - val_acc: 0.5000 - val_binary_crossentropy: 1.9554
Epoch 437/500
6/6 [==============================] - 1s 175ms/step - loss: 0.1378 - acc: 0.9583 - binary_crossentropy: 0.1378 - val_loss: 1.8324 - val_acc: 0.5417 - val_binary_crossentropy: 1.8136
Epoch 438/500
6

Epoch 449/500
6/6 [==============================] - 1s 178ms/step - loss: 0.0641 - acc: 0.9688 - binary_crossentropy: 0.0641 - val_loss: 1.7545 - val_acc: 0.5833 - val_binary_crossentropy: 1.7847
Epoch 450/500
6/6 [==============================] - 1s 178ms/step - loss: 0.0829 - acc: 0.9792 - binary_crossentropy: 0.0829 - val_loss: 2.1119 - val_acc: 0.5000 - val_binary_crossentropy: 2.0343
Epoch 451/500
6/6 [==============================] - 1s 177ms/step - loss: 0.1858 - acc: 0.9271 - binary_crossentropy: 0.1858 - val_loss: 1.8801 - val_acc: 0.5000 - val_binary_crossentropy: 1.8185
Epoch 452/500
6/6 [==============================] - 1s 176ms/step - loss: 0.0583 - acc: 0.9896 - binary_crossentropy: 0.0583 - val_loss: 1.8500 - val_acc: 0.5833 - val_binary_crossentropy: 1.8133
Epoch 453/500
6/6 [==============================] - 1s 176ms/step - loss: 0.0983 - acc: 0.9583 - binary_crossentropy: 0.0983 - val_loss: 1.7549 - val_acc: 0.5417 - val_binary_crossentropy: 1.8245
Epoch 454/500
6

Epoch 465/500
6/6 [==============================] - 1s 176ms/step - loss: 0.0593 - acc: 0.9688 - binary_crossentropy: 0.0593 - val_loss: 1.8245 - val_acc: 0.5417 - val_binary_crossentropy: 1.8771
Epoch 466/500
6/6 [==============================] - 1s 178ms/step - loss: 0.0988 - acc: 0.9688 - binary_crossentropy: 0.0988 - val_loss: 1.9785 - val_acc: 0.5000 - val_binary_crossentropy: 2.0183
Epoch 467/500
6/6 [==============================] - 1s 175ms/step - loss: 0.0593 - acc: 0.9688 - binary_crossentropy: 0.0593 - val_loss: 2.1358 - val_acc: 0.5417 - val_binary_crossentropy: 2.1790
Epoch 468/500
6/6 [==============================] - 1s 175ms/step - loss: 0.0774 - acc: 0.9479 - binary_crossentropy: 0.0774 - val_loss: 2.2922 - val_acc: 0.4583 - val_binary_crossentropy: 2.3145
Epoch 469/500
6/6 [==============================] - 1s 174ms/step - loss: 0.0866 - acc: 0.9688 - binary_crossentropy: 0.0866 - val_loss: 2.4663 - val_acc: 0.4583 - val_binary_crossentropy: 2.4882
Epoch 470/500
6

Epoch 481/500
6/6 [==============================] - 1s 176ms/step - loss: 0.2106 - acc: 0.9062 - binary_crossentropy: 0.2106 - val_loss: 1.6297 - val_acc: 0.4583 - val_binary_crossentropy: 1.6390
Epoch 482/500
6/6 [==============================] - 1s 176ms/step - loss: 0.1860 - acc: 0.9167 - binary_crossentropy: 0.1860 - val_loss: 1.8628 - val_acc: 0.5000 - val_binary_crossentropy: 1.8628
Epoch 483/500
6/6 [==============================] - 1s 176ms/step - loss: 0.0755 - acc: 0.9583 - binary_crossentropy: 0.0755 - val_loss: 1.6075 - val_acc: 0.4583 - val_binary_crossentropy: 1.6182
Epoch 484/500
6/6 [==============================] - 1s 177ms/step - loss: 0.0843 - acc: 0.9688 - binary_crossentropy: 0.0843 - val_loss: 1.4938 - val_acc: 0.4167 - val_binary_crossentropy: 1.5005
Epoch 485/500
6/6 [==============================] - 1s 178ms/step - loss: 0.0739 - acc: 0.9792 - binary_crossentropy: 0.0739 - val_loss: 1.6957 - val_acc: 0.3750 - val_binary_crossentropy: 1.6511
Epoch 486/500
6

Epoch 497/500
6/6 [==============================] - 1s 177ms/step - loss: 0.1068 - acc: 0.9479 - binary_crossentropy: 0.1068 - val_loss: 1.6920 - val_acc: 0.5417 - val_binary_crossentropy: 1.8538
Epoch 498/500
6/6 [==============================] - 1s 177ms/step - loss: 0.1428 - acc: 0.9583 - binary_crossentropy: 0.1428 - val_loss: 1.7719 - val_acc: 0.5417 - val_binary_crossentropy: 1.9432
Epoch 499/500
6/6 [==============================] - 1s 178ms/step - loss: 0.0272 - acc: 0.9896 - binary_crossentropy: 0.0272 - val_loss: 1.9051 - val_acc: 0.5417 - val_binary_crossentropy: 2.0571
Epoch 500/500
24/24 [==============================] - 0s 3ms/sample - loss: 1.7194 - acc: 0.4583 - binary_crossentropy: 1.7194


In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
dataset_loader = DatasetLoader(f"{slice_dataset}/dataset-{dataset}-{sequence}-{side}-{percentile}-perc.pickle")

count_created_data(dataset, epochs=500, batch_sizes=16)

Batch: 16
X shape (96, 224, 224, 1)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,) (96,)
(96,

KeyboardInterrupt: 

In [58]:
#scores_str = str(scores * 100)
#scores_str

In [66]:
print(str(train_score[1]),
str(train_score[0]),
str(test_score[1]),
str(test_score[0]))

0.8958333 0.2194067786137263 0.45833334 1.7193984985351562


In [44]:
import matplotlib.pyplot as plt

acc = fit.history['acc']
val_acc = fit.history['val_acc']

loss = fit.history['loss']
val_loss = fit.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
# plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

NameError: name 'fit' is not defined